In [1]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import pickle
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("synthetic")

LangSmith 추적을 시작합니다.
[프로젝트명]
synthetic


In [4]:
with open("../../outputs/sample_output_split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)

In [5]:
len(split_documents)

108

In [8]:
split_documents[2].page_content

'This work is published under the responsibility of the Secretary-General of the OECD. The opinions expressed and  \narguments employed herein do not necessarily reflect the official views of the Member countries of the OECD.\n\nThis work is published under the responsibility of the Secretary-General of the OECD. The opinions expressed and  \narguments employed herein do not necessarily reflect the official views of the Member countries of the OECD.\n\nThis document, as well as any data and map included herein, are without prejudice to the status of or sovereignty over  \nany territory, to the delimitation of international frontiers and boundaries and to the name of any territory, city or area.\n\nThe statistical data for Israel are supplied by and under the responsibility of the relevant Israeli authorities. The use of  \nsuch data by the OECD is without prejudice to the status of the Golan Heights, East Jerusalem and Israeli settlements in  \nthe West Bank under the terms of internat

In [5]:
llm = ChatOpenAI(model_name="gpt-5-nano", temperature=0)

In [6]:
class SyntheticData(BaseModel):
    """Synthetic data with details."""
    query: str = Field(..., description="The query of the data")
    answers: str = Field(..., description="The answers of the data")
    chunk_id: str = Field(..., description="The chunk id of the evidence")
    page_number: str = Field(..., description="The page number of the evidence")
    evidence: str = Field(..., description="The evidence of the data")
 
 

In [7]:
trimmed = split_documents[2:]

In [8]:
len(trimmed), len(split_documents)

(107, 109)

In [9]:
system_prompt = "You are a careful dataset generator for RAG. Only answer from the provided passage."
    
user_prompt = f"""
            Task: Extract 3 atomic facts and write 1 QA whose answer from following document 
            Response: query : question, answers : answer, evidence : page content for the answer
        """

queries = []
for doc in trimmed:
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
        {"role": "user", "content": "chunk id: " + str(doc.metadata["id"])},
        {"role": "user", "content": "page number: " + str(doc.metadata["page"])},
        {"role": "user", "content": "content : " + doc.page_content},
    ]
    queries.append(prompt)



In [10]:
model_with_structure = llm.with_structured_output(SyntheticData)
responses = model_with_structure.batch(queries)

In [11]:
len(responses)

107

In [17]:
arr = []
for r in responses:
    obj = {
        "query": r.query,
        "answers": r.answers,
        "evidence": r.evidence,
        "page_number": r.page_number,
        "chunk_id": r.chunk_id,
    }
    arr.append(obj)



In [18]:
with open("outputs/synthetic_out.jsonl", "w", encoding="utf-8") as f:
    for r in arr:
        f.write(json.dumps(r, ensure_ascii=False, indent=2) + "\n")

In [19]:
import pandas as pd
df = pd.DataFrame(arr)
df.to_csv("outputs/synthetic_out.csv", index=True)

In [15]:
# To get 'query' and 'answers' in a specific row (e.g., row number i), use:
i = 15  # Change this to your desired row index
query = df.loc[i, "query"]
answers = df.loc[i, "answers"]
print("Query:", query)
print("Answers:", answers)


Query: What are three key policy recommendations mentioned in the document?
Answers: 1) Countries should engage cooperatively within the global trading system and make trade policy more transparent and predictable while addressing economic security concerns. 2) Central banks should remain vigilant and react promptly to shifts in the balance of risks to price stability. 3) Effective monitoring, supervision, and robust regulatory policies are needed to address financial stability risks.
